<a href="https://colab.research.google.com/github/vitroid/ClustIce/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
try:
    import google.colab
    ! pip install py3Dmol genice-core genice2 git+https://github.com/vitroid/clustice.git
except:
    pass

In [ ]:
from logging import getLogger, INFO, DEBUG
import numpy as np
import networkx as nx
import py3Dmol

import genice_core
from clustice.serialize import serialize
from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.water import tip4p, spce
from clustice import graph
from cycless import cycles

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

# note: g must be a graph whose labels start from 0.
g = graph.great_icosahedron(8, separation=L)
# g = graph.great_decahedron(4)
# g = graph.small_barrelan()
# g = graph.large_barrelan()
# g = graph.twistane()
# g = graph.adamantane()
# g = nx.cycle_graph(6) # hexagon
# g = nx.cycle_graph(7) # heptagon
# g = nx.cubical_graph() # cubic octamer
# g = nx.dodecahedral_graph()

# count cycles in the graph
cyclehist = {}
for cycle in cycles.cycles_iter(g, maxsize=5):
    nnode = len(cycle)
    cyclehist[nnode] = cyclehist.get(nnode, 0) + 1
print(cyclehist)


if "pos" in g.nodes[0]:
    # extract the embedded coords in g
    mol_positions = np.array([g.nodes[v]["pos"] for v in g])
else:
    # estimate of the positions of the nodes
    mol_positions = make_layout(g, edge_length=L)

# set orientations of the hydrogen bonds.
# if pos is given, the net dipole moment is minimized.
dg = genice_core.ice_graph(
    g, vertexPositions=mol_positions, dipoleOptimizationCycles=1000
)
# dg = ice_graph(g)

# get the unique id for the graph
# id = serialize(dg)
# print(id)

# put water molecules
gro = render(
    dg,
    mol_positions,
    water_model=spce,
    cell_matrix=np.array([[30.0, 0.0, 0.0], [0.0, 30.0, 0.0], [0.0, 0.0, 30.0]]),
    shift=np.array([0.5, 0.5, 0.5]),
)
with open("save.gro", "w") as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})  #: {"radius": 0.75}})
view.addUnitCell()
view.zoomTo()
view.show()
view.png()

{5: 108}


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Rebuild the cluster from id.


In [ ]:
import networkx as nx

from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.serialize import deserialize
from clustice.water import tip4p

import py3Dmol

s = "1.2.3+6.19.3.4+6.7.14.1+7.8+13.0+9.10+12.16.12.13+15.5.15+17.4+18.11.0+18"

dg = deserialize(s)

# Make layout of the nodes
g = nx.Graph(dg)
L = 0.27
mol_positions = make_layout(g, edge_length=L)

# put water molecules
gro = render(dg, mol_positions, water_model=tip4p)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

In [1]:
import networkx as nx

from genice2.genice import GenIce
from genice2.plugin import Format, Lattice
from clustice.gromacs import render
from clustice.water import tip4p
import genice_core

import py3Dmol

lattice = Lattice("ice1h")
formatter = Format("raw", stage=(1, 2))
raw = GenIce(lattice, signature="Ice Ih", rep=(3, 3, 3)).generate_ice(formatter)

# graph is the topology of the hydrogen-bond network
g = nx.Graph(raw["graph"])
# reppositions contains the positions of CoM of water in fractional coordinate
mol_positions = raw["reppositions"]
# repcell is the cell matrix (transposed)
cell_matrix = raw["repcell"]

# set orientations of the hydrogen bonds.
# It automatically depolarizes by optimizing the arrangement.
dg = genice_core.ice_graph(
    g,
    vertexPositions=mol_positions,
    isPeriodicBoundary=True,
    dipoleOptimizationCycles=200,
)

# put water molecules
gro = render(
    dg,
    mol_positions @ cell_matrix,
    water_model=tip4p,
    cell_matrix=cell_matrix,
)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 3D Object


In [5]:
! pip install viewscad solidpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 16.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.7/271.7 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PrettyTable: filename=prettytable-0.7.2-py3-none-any.whl size=13696 sha256=1701e793f492116a4c12ac1f6630a07269d2fe42ae6dfac7741baa263fffaa24
  Stored in directory: /Users/matto/Library/Caches/pip/wheels/b1/e4/65/051f9bd54c89d377db5ca942a672439c36cbf5ceff8e8902e0
  Created wheel for euclid3: filename=euclid3-0.1-py3-none-any.whl size=13228 sha256=ce099b778a89607fca1a17da0fd36d1fa4cc66972371fb890a0734d604db30ce
  Stored in directory: /U

In [1]:
from logging import getLogger, INFO, DEBUG
import numpy as np
import networkx as nx

import genice_core
from clustice.solid import model
from clustice.water import tip4p
from clustice import graph
from solid2 import scad_render_to_file, scale

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

size = 2
# note: g must be a graph whose labels start from 0.
# g = graph.great_icosahedron(1, separation=L)
g = graph.great_decahedron(size)
# g = graph.small_barrelan()
# g = graph.large_barrelan()
# g = graph.twistane()
# g = graph.adamantane()
# g = nx.cycle_graph(6) # hexagon
# g = nx.cycle_graph(7) # heptagon
# g = nx.cubical_graph() # cubic octamer
# g = nx.dodecahedral_graph()

if "pos" in g.nodes[0]:
    # extract the embedded coords in g
    mol_positions = np.array([g.nodes[v]["pos"] for v in g])
else:
    # estimate of the positions of the nodes
    mol_positions = make_layout(g, edgelen=L)


for vr, er in (
    (0.2, 0.0),
    (0.15, 0.0),
    (0.1, 0.0),
    (0.05, 0.0),
    (0.0, 0.1),
    (0.0, 0.07),
    (0.0, 0.03),
):
    # put water molecules
    m = model(
        g,
        mol_positions,
        water_model=tip4p,
        cell_matrix=np.array([[10.0, 0.0, 0.0], [0.0, 10.0, 0.0], [0.0, 0.0, 10.0]]),
        vertex_radius=vr,
        edge_radius=er,
        # shift=np.array([0.5, 0.5, 0.5]),
    )

    scad_render_to_file(m.scale(100), f"5bp{size}_{vr}_{er}.scad")

0 220
0 55
0 35
1 221
1 56
1 35
1 36
2 222
2 57
2 36
2 37
3 223
3 58
3 37
3 38
4 224
4 59
4 38
5 35
5 39
5 70
6 39
6 36
6 40
6 71
7 37
7 40
7 41
7 72
8 38
8 41
8 73
9 39
9 80
9 42
10 40
10 81
10 42
10 43
11 41
11 82
11 43
12 42
12 86
12 44
13 43
13 87
13 44
14 44
14 89
15 225
15 45
15 35
16 226
16 45
16 46
16 36
17 227
17 46
17 47
17 37
18 228
18 47
18 38
19 45
19 39
19 48
20 46
20 40
20 48
20 49
21 47
21 41
21 49
22 48
22 42
22 50
23 49
23 43
23 50
24 50
24 44
25 229
25 45
25 51
26 230
26 46
26 51
26 52
27 231
27 47
27 52
28 51
28 48
28 53
29 52
29 49
29 53
30 53
30 50
31 232
31 51
31 54
32 233
32 52
32 54
33 54
33 53
34 234
34 54
55 110
55 90
56 111
56 90
56 91
57 112
57 91
57 92
58 113
58 92
58 93
59 114
59 93
60 125
60 90
60 94
61 126
61 91
61 94
61 95
62 127
62 92
62 95
62 96
63 128
63 93
63 96
64 135
64 97
64 94
65 136
65 97
65 98
65 95
66 137
66 98
66 96
67 141
67 97
67 99
68 142
68 98
68 99
69 144
69 99
70 90
70 100
71 91
71 100
71 101
72 92
72 101
72 102
73 93
73 102
74 94
74 

'/Users/matto/Dropbox/gitbox/ClustIce/5bp2_0.1_0.05.scad'

# Benchmark tests


In [ ]:
from clustice.geometry import make_layout
from clustice.graph import great_icosahedron

from logging import getLogger, INFO

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

# note: g must be a graph whose labels start from 0.
# g = nx.dodecahedral_graph() # dodecahedral 20mer
g = great_icosahedron(1)
mol_positions = make_layout(g, edge_length=L)
print(len(mol_positions))

In [ ]:
# estimate of the positions of the nodes
mol_positions = make_layout(g, edge_length=L)

In [ ]:
dg = genice_core.ice_graph(g, vertexPositions=mol_positions)

In [ ]:
dg = genice_core.ice_graph(g)

In [ ]:
# great_icosahedron
# size  nnode   constellation        depolarization       ice rule
benchmark = [
    [0, 20, 0.146, 0.00612, 0.000410],
    [1, 100, 0.676, 0.00865, 0.00229],
    [2, 280, 3.14, 0.0178, 0.00569],
    [3, 600, 51.4, 0.024, 0.0133],
]